# Import Libraries

In [33]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/mira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mira/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load the Data

In [34]:
df = pd.read_csv('news_dataset.csv')
documents = df['text'].tolist()

# Preprocess the Data

In [35]:
# Create a set of English stopwords
stop_words = set(stopwords.words('english'))

# Initialize a WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)  # Convert non-string inputs to string
    tokens = word_tokenize(text.lower())  # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize each token
    tokens = [token for token in tokens if len(token) > 1]  # Remove single alphabet letters
    tokens = [token for token in tokens if not token.isdigit()]  # Remove numbers
    tokens = [token for token in tokens if re.match(r'^[a-zA-Z]+$', token)]  # Remove any remaining special characters
    return tokens  # Return the preprocessed tokens

# Preprocess each document in the list, handling potential non-string entries
preprocessed_documents = [preprocess_text(doc) for doc in documents]

print(preprocessed_documents[0])

['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'sport', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please']


# Create document-term matrix

In [36]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

# Filter out tokens that appear in less than 15 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=15, no_above=0.5)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

# Run LDA

In [37]:
# Run LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) # Train an LDA modelon the corpus with 2 topics using Gensim's

# Evaluate the LDA model using Coherence score

In [38]:
# Calculate the coherence score for the LDA model
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocessed_documents, dictionary=dictionary, coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
# Display the score

print(f'Topic Coherence Score (C_V): {coherence_lda:.4f}')


Topic Coherence Score (C_V): 0.5258


# Interpret Results

In [39]:
# empty list to store dominant topic labels for each document
text_labels = []

# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to bag-of-words representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # append to the list
    text_labels.append(dominant_topic)
    
# Create DataFrame
df_result = pd.DataFrame({"Text": documents, "Topic": text_labels})

# Print the DataFrame
print("Table with Text and Topic:")
print(df_result)
print()

Table with Text and Topic:
                                                    Text  Topic
0      I was wondering if anyone out there could enli...      3
1      I recently posted an article asking what kind ...      3
2      \nIt depends on your priorities.  A lot of peo...      3
3      an excellent automatic can be found in the sub...      3
4      : Ford and his automobile.  I need information...      3
...                                                  ...    ...
11309  Secrecy in Clipper Chip\n\nThe serial number o...      2
11310  Hi !\n\nI am interested in the source of FEAL ...      3
11311  The actual algorithm is classified, however, t...      2
11312  \n\tThis appears to be generic calling upon th...      4
11313  \nProbably keep quiet and take it, lest they g...      3

[11314 rows x 2 columns]



In [40]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top terms for Topic #0:
['max', 'armenian', 'government', 'people', 'state', 'said', 'one', 'right', 'turkish', 'war']

Top terms for Topic #1:
['game', 'db', 'team', 'year', 'new', 'player', 'president', 'play', 'university', 'first']

Top terms for Topic #2:
['key', 'file', 'use', 'system', 'program', 'information', 'encryption', 'chip', 'available', 'data']

Top terms for Topic #3:
['one', 'would', 'get', 'like', 'know', 'drive', 'problem', 'use', 'thanks', 'work']

Top terms for Topic #4:
['would', 'one', 'people', 'think', 'know', 'say', 'like', 'god', 'time', 'make']



In [41]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
- "max" (weight: 0.022)
- "armenian" (weight: 0.011)
- "government" (weight: 0.011)
- "people" (weight: 0.008)
- "state" (weight: 0.007)
- "said" (weight: 0.006)
- "one" (weight: 0.005)
- "right" (weight: 0.005)
- "turkish" (weight: 0.005)
- "war" (weight: 0.005)

Topic 1:
- "game" (weight: 0.013)
- "db" (weight: 0.013)
- "team" (weight: 0.011)
- "year" (weight: 0.011)
- "new" (weight: 0.007)
- "player" (weight: 0.007)
- "president" (weight: 0.006)
- "play" (weight: 0.006)
- "university" (weight: 0.006)
- "first" (weight: 0.005)

Topic 2:
- "key" (weight: 0.016)
- "file" (weight: 0.012)
- "use" (weight: 0.010)
- "system" (weight: 0.009)
- "program" (weight: 0.008)
- "information" (weight: 0.007)
- "encryption" (weight: 0.007)
- "chip" (weight: 0.007)
- "available" (weight: 0.006)
- "data" (weight: 0.005)

Topic 3:
- "one" (weight: 0.010)
- "would" (weight: 0.009)
- "get" (weight: 0.009)
- "like" (weight: 0.008)
- "know" (weight: 0.007)
- "drive" (weig